In [1]:
import os
import subprocess
import time
import ssl
import certifi
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow

# Google API configuration (Now uses client_secret.json)
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"  # ✅ Now uses the same JSON file

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    Uses TLS protocol compatibility to avoid SSL issues.
    """
    # Authenticate with OAuth 2.0 (like the list videos script)
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    creds = flow.run_local_server(port=0)
    return build('youtube', 'v3', credentials=creds)

def merge_videos_and_create_timestamps(folder_path):
    """
    Merges MP4 videos in a folder into one video and creates a timestamp file.
    Returns the merged video path and description content (timestamps).
    """
    if not os.path.exists(folder_path):
        print(f"Error: The folder path '{folder_path}' does not exist.")
        return None, None

    # Gather video files
    video_files = []
    timestamps = []
    total_duration = 0

    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(".mp4"):
            filepath = os.path.join(folder_path, filename)
            video_files.append(filepath)

            try:
                probe = subprocess.run(
                    ["ffprobe", "-v", "error", "-show_entries", "format=duration",
                     "-of", "default=noprint_wrappers=1:nokey=1", filepath],
                    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
                )
                duration = float(probe.stdout.strip())
                timestamps.append((filename, total_duration))
                total_duration += duration
            except Exception as e:
                print(f"Error reading duration for file '{filename}': {e}")

    if not video_files:
        print("No MP4 files found.")
        return None, None

    # Create merged video name from last 4 folders in the path
    folder_parts = os.path.normpath(folder_path).split(os.sep)
    merged_video_name = " - ".join(folder_parts[-4:]) + ".mp4" if len(folder_parts) >= 4 else "merged_video.mp4"
    merged_video_path = os.path.join(folder_path, merged_video_name)

    # Prepare file list for FFmpeg
    file_list_path = os.path.join(folder_path, "file_list.txt")
    with open(file_list_path, "w") as f:
        for video_file in video_files:
            f.write(f"file '{video_file}'\n")

    # Merge videos using FFmpeg
    try:
        print("Merging videos...")
        subprocess.run(
            ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, "-c", "copy", merged_video_path],
            check=True
        )
    except subprocess.CalledProcessError as e:
        print(f"Error during merging: {e}")
        return None, None

    # Generate timestamps in HH:MM:SS format
    timestamp_file_path = os.path.join(folder_path, "timestamps.txt")
    with open(timestamp_file_path, "w") as f:
        description = ""
        for filename, timestamp in timestamps:
            hours = int(timestamp // 3600)
            minutes = int((timestamp % 3600) // 60)
            seconds = int(timestamp % 60)
            timestamp_entry = f"{filename} - {hours:02d}:{minutes:02d}:{seconds:02d}"
            f.write(timestamp_entry + "\n")
            description += timestamp_entry + "\n"

    print(f"Merged video saved to: {merged_video_path}")
    print(f"Timestamps saved to: {timestamp_file_path}")
    return merged_video_path, description

def upload_to_youtube(video_path, title, description):
    """
    Uploads a video to YouTube.
    """
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": ["training", "tutorial", "video"],  # Add any tags you want
                "categoryId": "27"  # Education category
            },
            "status": {
                "privacyStatus": "unlisted"  # Change to "public" or "private" as needed
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    print("Uploading video to YouTube...")
    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")
        except Exception as e:
            raise Exception(f"Error during upload: {e}")
    print(f"✅ Video uploaded successfully! Video ID: {response['id']}")

def upload_to_youtube_with_retry(video_path, title, description, retries=3, delay=10):
    """
    Uploads a video to YouTube with retry logic.
    """
    for attempt in range(retries):
        try:
            upload_to_youtube(video_path, title, description)
            return
        except Exception as e:
            print(f"Error during upload: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("❌ Upload failed after multiple attempts.")

if __name__ == "__main__":
    folder_path = input("Enter the folder path containing the videos: ").strip()

    # Step 1: Merge videos and create timestamps
    merged_video_path, description = merge_videos_and_create_timestamps(folder_path)

    # Step 2: Upload merged video to YouTube with retry logic
    if merged_video_path and description:
        title = os.path.basename(merged_video_path).replace(".mp4", "")  # Use the merged video name as the title
        upload_to_youtube_with_retry(merged_video_path, title, description)


Merging videos...
Merged video saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\InDesign\Lynda\InDesign CC 2018 - Essential Training\Adobe - InDesign - Lynda - InDesign CC 2018 - Essential Training.mp4
Timestamps saved to: H:\Projects Control (PC)\10 Backup\05 Tutorials\Adobe\InDesign\Lynda\InDesign CC 2018 - Essential Training\timestamps.txt
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60968%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=1FVunMnsO2HgXySYKvfZEHSpjOWO4x&access_type=offline
Uploading video to YouTube...
✅ Video uploaded successfully! Video ID: CWrCCN8smxY
